# Example usage notebook
Objects and functions in this notebook are listed with all paramaters to ilustrate their capabillities. Most of the paramaters have default values in the implementation

## Imports

In [1]:
import os
import sys

sys.path.append(os.getcwd())
os.chdir("..")

import pandas as pd
import mlflow

pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 300)

In [2]:
import os
import json
from typing import Literal
import pandas as pd

from churn_pred.preprocessing.preprocess import PreprocessData
from churn_pred.training.trainer import Trainer
from churn_pred.training.optuna_optimizer import LGBOptunaOptimizer
from churn_pred.training.utils import flatten_dict, get_or_create_experiment

import dill
import numpy as np
from churn_pred.utils import dill_dump, dill_load
from sklearn.model_selection import train_test_split

from pprint import pprint

## Train a testing model 

In [3]:
# 1. get the data
df_pd = pd.read_parquet("data/dataset_auxiliary_features_cleaned.parquet")
df_pd.head()

,CustomerId,CreditScore,Country,Gender,Age,Tenure,Balance (EUR),NumberOfProducts,HasCreditCard,IsActiveMember,EstimatedSalary,Exited,CustomerFeedback_sentiment3,CustomerFeedback_sentiment5,Surname_Country,Surname_Country_region,Surname_Country_subregion,Country_region,Country_subregion,is_native,Country_hemisphere,Country_gdp_per_capita,Country_IncomeGroup,Surname_Country_gdp_per_capita,Surname_Country_IncomeGroup,working_class,stage_of_life,generation
0,15787619,844,France,Male,18,2,160980.03,1,0,0,145936.28,0,neutral,4 stars,Taiwan,Asia,Eastern Asia,Europe,Western Europe,0,northern,57594.03402,High income,32756.00000,None,working_age,teen,gen_z
1,15770309,656,France,Male,18,10,151762.74,1,0,1,127014.32,0,neutral,1 star,United States,Americas,Northern America,Europe,Western Europe,0,northern,57594.03402,High income,76329.58227,High income,working_age,teen,gen_z
2,15569178,570,France,Female,18,4,82767.42,1,1,0,71811.90,0,neutral,2 stars,Russian Federation,Europe,Eastern Europe,Europe,Western Europe,0,northern,57594.03402,High income,34637.76172,Upper middle income,working_age,teen,gen_z
3,15795519,716,Germany,Female,18,3,128743.80,1,0,0,197322.13,0,neutral,2 stars,Russian Federation,Europe,Eastern Europe,Europe,Western Europe,0,northern,66616.02225,High income,34637.76172,Upper middle income,working_age,teen,gen_z
4,15621893,727,France,Male,18,4,133550.67,1,1,1,46941.41,0,positive,1 star,Italy,Europe,Southern Europe,Europe,Western Europe,0,northern,57594.03402,High income,55442.07843,High income,working_age,teen,gen_z


In [4]:
target_col = "Exited"
id_cols = ["CustomerId"]
cat_cols = ["Country", "Gender", "HasCreditCard", "IsActiveMember","CustomerFeedback_sentiment3", "CustomerFeedback_sentiment5", "Surname_Country", "Surname_Country_region", "Surname_Country_subregion", "Country_region", "Country_subregion", "is_native", "Country_hemisphere", "Country_IncomeGroup", "Surname_Country_IncomeGroup", "working_class", "stage_of_life", "generation"]
cont_cols = df_pd.drop(columns=id_cols + cat_cols + [target_col]).columns.values.tolist()

In [5]:
df_pd[cat_cols] = df_pd[cat_cols].astype(str)

In [28]:
valid_size = 0.2
test_size = 0.5
random_state = 1
df_train, df_valid = train_test_split(df_pd, test_size=valid_size, stratify=df_pd[target_col], random_state=random_state)
df_valid, df_test = train_test_split(df_valid, test_size=test_size, stratify=df_valid[target_col], random_state=random_state)

In [7]:
prepare_data = PreprocessData(
        id_cols=id_cols,
        target_col=target_col,
        cat_cols=cat_cols,
        cont_cols=cont_cols,
    )
# this should be fitted only on training data
_ = prepare_data.fit(df=df_pd)

In [8]:
optimizer = LGBOptunaOptimizer(
        objective="binary",
        n_class=2,
    )

trainer = Trainer(
        cat_cols=prepare_data.cat_cols,
        target_col=prepare_data.target_col,
        id_cols=id_cols,
        objective="binary",
        n_class=2,
        optimizer=optimizer,
        preprocessors=[prepare_data],
)

metrics_dict = trainer.fit(
    df_train=df_train,
    df_valid=df_valid,
    df_test=df_test,
    )

[I 2024-06-18 21:05:47,850] A new study created in memory with name: no-name-72ade7dc-1867-41c2-b5a4-df3b58765bc7
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 50 rounds


feature_fraction, val_score: 0.388064:  14%|#4        | 1/7 [00:08<00:48,  8.11s/it]

Early stopping, best iteration is:
[241]	valid_0's binary_logloss: 0.388064
Training until validation scores don't improve for 50 rounds


feature_fraction, val_score: 0.384552:  29%|##8       | 2/7 [00:15<00:39,  7.81s/it]

...

Early stopping, best iteration is:
[113]	valid_0's binary_logloss: 0.393828
Training until validation scores don't improve for 50 rounds


min_child_samples, val_score: 0.364819:  80%|########  | 4/5 [00:53<00:13, 13.86s/it]

Early stopping, best iteration is:
[32]	valid_0's binary_logloss: 0.367124
Training until validation scores don't improve for 50 rounds


min_child_samples, val_score: 0.364819: 100%|##########| 5/5 [01:07<00:00, 13.50s/it]

Early stopping, best iteration is:
[38]	valid_0's binary_logloss: 0.371929


In [8]:
# save/load trainer
# dill_dump("lgbm_trainer.dill", trainer)
trainer = dill_load("lgbm_trainer.dill")

In [11]:
pprint(metrics_dict)

{'cls_report': {'0': {'f1-score': 0.9244823386114495,
                      'precision': 0.8971631205673759,
                      'recall': 0.9535175879396985,
                      'support': 796},
                '1': {'f1-score': 0.6536312849162011,
                      'precision': 0.7597402597402597,
                      'recall': 0.5735294117647058,
                      'support': 204},
                'accuracy': 0.876,
                'macro avg': {'f1-score': 0.7890568117638252,
                              'precision': 0.8284516901538178,
                              'recall': 0.7635234998522022,
                              'support': 1000},
                'weighted avg': {'f1-score': 0.8692287236576187,
                                 'precision': 0.8691288569586443,
                                 'recall': 0.876,
                                 'support': 1000}},
 'cm': [[759, 37], [87, 117]],
 'prec_rec_curve': [[0.204, 0.7597402597402597, 1.0],
              

In [32]:
pprint(trainer.compute_metrics(df_test, with_dynamic_binary_threshold=True))

{'cls_report': {'0': {'f1-score': 0.9244823386114495,
                      'precision': 0.8971631205673759,
                      'recall': 0.9535175879396985,
                      'support': 796},
                '1': {'f1-score': 0.6536312849162011,
                      'precision': 0.7597402597402597,
                      'recall': 0.5735294117647058,
                      'support': 204},
                'accuracy': 0.876,
                'macro avg': {'f1-score': 0.7890568117638252,
                              'precision': 0.8284516901538178,
                              'recall': 0.7635234998522022,
                              'support': 1000},
                'weighted avg': {'f1-score': 0.8692287236576187,
                                 'precision': 0.8691288569586443,
                                 'recall': 0.876,
                                 'support': 1000}},
 'cm': [[759, 37], [87, 117]],
 'prec_rec_curve': [[0.204, 0.7597402597402597, 1.0],
              

# Mlflow
* tracking
* model registration in Minio

In [12]:
mlflow_host = "10.152.183.54"
mlflow_host_url = "mlflow.mlflow.svc.cluster.local"
mlflow_port = "5000"
os.environ["AWS_ACCESS_KEY_ID"] = "minioadmin"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minioadmin"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = f"http://10.152.183.156:9000"

mlflow.set_tracking_uri("http://" + mlflow_host + ":" + mlflow_port)
experiment_id = get_or_create_experiment("ecovadis")
mlflow.set_experiment(experiment_id=experiment_id)

<Experiment: artifact_location='s3://mlflow/6', creation_time=1718745770093, experiment_id='6', last_update_time=1718745770093, lifecycle_stage='active', name='ecovadis', tags={}>

In [13]:
metrics_dict_flattened = flatten_dict(metrics_dict)

In [14]:
# mlflow metrics can be only int, float not list
del metrics_dict_flattened["cm"]
del metrics_dict_flattened["prec_rec_curve"]

In [15]:
run_name = "init_run"
with mlflow.start_run(experiment_id=experiment_id, run_name=run_name, nested=True) as run:
    mlflow.log_params(trainer.optimizer.best)
    mlflow.log_metrics(metrics_dict_flattened)

    # Log tags
    mlflow.set_tags(
        tags={
            "project": "ecovadis",
            "optimizer_engine": "optuna",
            "model_family": "ligtgbm",
            "feature_set_version": 1,
        }
    )
    # Log figure - for future fun
    # mlflow.log_figure(figure=correlation_plot, artifact_file="correlation_plot.png")

    artifact_path = "ecovadis_model"
    registered_model_name = "ecovadis_lgbm_model"
    mlflow.pyfunc.log_model(python_model=trainer, 
                            artifact_path=artifact_path, 
                            registered_model_name=registered_model_name,
                            code_paths=["churn_pred"])  
    model_uri = mlflow.get_artifact_uri(artifact_path)
    print(f"Run ID:\n{run.info.run_id}\nModel uri:\n{model_uri}")

2024/06/18 21:23:54 INFO mlflow.types.utils: Unsupported type hint: <class 'pandas.core.frame.DataFrame'>, skipping schema inference
2024/06/18 21:23:54 INFO mlflow.types.utils: Unsupported type hint: <class 'pandas.core.frame.DataFrame'>, skipping schema inference
2024/06/18 21:23:58 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpfsv2c_xg/model, flavor: python_function). Fall back to return ['cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback. 
Successfully registered model 'ecovadis_lgbm_model'.
2024/06/18 21:27:30 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ecovadis_lgbm_model, version 1


Run ID:
761906b692e94248940c6dbf6b12c23c
Model uri:
s3://mlflow/6/761906b692e94248940c6dbf6b12c23c/artifacts/ecovadis_model


Created version '1' of model 'ecovadis_lgbm_model'.


## Screenshots from MLflow GUI (Nodeport service in Kubernetes)

![mlflow_parameters](img/mlflow_parameters.png)
![mlflow_parameters](img/mlflow_metrics.png)

![mlflow_parameters](img/mlflow_artifacts.png)

![mlflow_parameters](img/mlflow_artifacts_registered_model.png)

## Predictions testing

**IMPORTANT**: the predictions are in raw_score format(for testing to see if I get same predictions), i.e. strange numbers and not classes in the output

### Example data

In [16]:
df_pd.iloc[:2]

,CustomerId,CreditScore,Country,Gender,Age,Tenure,Balance (EUR),NumberOfProducts,HasCreditCard,IsActiveMember,EstimatedSalary,Exited,CustomerFeedback_sentiment3,CustomerFeedback_sentiment5,Surname_Country,Surname_Country_region,Surname_Country_subregion,Country_region,Country_subregion,is_native,Country_hemisphere,Country_gdp_per_capita,Country_IncomeGroup,Surname_Country_gdp_per_capita,Surname_Country_IncomeGroup,working_class,stage_of_life,generation
0,15787619,844,France,Male,18,2,160980.03,1,0,0,145936.28,0,neutral,4 stars,Taiwan,Asia,Eastern Asia,Europe,Western Europe,0,northern,57594.03402,High income,32756.00000,None,working_age,teen,gen_z
1,15770309,656,France,Male,18,10,151762.74,1,0,1,127014.32,0,neutral,1 star,United States,Americas,Northern America,Europe,Western Europe,0,northern,57594.03402,High income,76329.58227,High income,working_age,teen,gen_z


In [16]:
df_pd.iloc[:2].transpose().to_dict(orient='index')

{'CustomerId': {0: 15787619, 1: 15770309},
 'CreditScore': {0: 844, 1: 656},
 'Country': {0: 'France', 1: 'France'},
 'Gender': {0: 'Male', 1: 'Male'},
 'Age': {0: 18, 1: 18},
 'Tenure': {0: 2, 1: 10},
 'Balance (EUR)': {0: 160980.03, 1: 151762.74},
 'NumberOfProducts': {0: 1, 1: 1},
 'HasCreditCard': {0: '0', 1: '0'},
 'IsActiveMember': {0: '0', 1: '1'},
 'EstimatedSalary': {0: 145936.28, 1: 127014.32},
 'Exited': {0: 0, 1: 0},
 'CustomerFeedback_sentiment3': {0: 'neutral', 1: 'neutral'},
 'CustomerFeedback_sentiment5': {0: '4 stars', 1: '1 star'},
 'Surname_Country': {0: 'Taiwan', 1: 'United States'},
 'Surname_Country_region': {0: 'Asia', 1: 'Americas'},
 'Surname_Country_subregion': {0: 'Eastern Asia', 1: 'Northern America'},
 'Country_region': {0: 'Europe', 1: 'Europe'},
 'Country_subregion': {0: 'Western Europe', 1: 'Western Europe'},
 'is_native': {0: '0', 1: '0'},
 'Country_hemisphere': {0: 'northern', 1: 'northern'},
 'Country_gdp_per_capita': {0: 57594.03402, 1: 57594.03402},

after some notepad magic:
```
{
    "CustomerId": [15787619, 15770309],
    "CreditScore": [844, 656],
    "Country": ["France", "France"],
    "Gender": ["Male", "Male"],
    "Age": [18, 18],
    "Tenure": [2, 10],
    "Balance (EUR)": [160980.03, 151762.74],
    "NumberOfProducts": [1, 1],
    "HasCreditCard": ["0", "0"],
    "IsActiveMember": ["0", "1"],
    "EstimatedSalary": [145936.28, 127014.32],
    "Exited": [0, 0],
    "CustomerFeedback_sentiment3": ["neutral", "neutral"],
    "CustomerFeedback_sentiment5": ["4 stars", "1 star"],
    "Surname_Country": ["Taiwan", "United States"],
    "Surname_Country_region": ["Asia", "Americas"],
    "Surname_Country_subregion": ["Eastern Asia", "Northern America"],
    "Country_region": ["Europe", "Europe"],
    "Country_subregion": ["Western Europe", "Western Europe"],
    "is_native": ["0", "0"],
    "Country_hemisphere": ["northern", "northern"],
    "Country_gdp_per_capita": [57594.03402, 57594.03402],
    "Country_IncomeGroup": ["High income", "High income"],
    "Surname_Country_gdp_per_capita": [32756.0, 76329.58227],
    "Surname_Country_IncomeGroup": ["None", "High income"],
    "working_class": ["working_age", "working_age"],
    "stage_of_life": ["teen", "teen"],
    "generation": ["gen_z", "gen_z"],
}
```

### Trainer

In [16]:
trainer.predict(df=df_pd.iloc[:2].drop(columns=["Exited"]), context={})

,Exited
0,-9.958064
1,-12.560959


### Downloaded Trainer

In [17]:
model_uri

's3://mlflow/6/761906b692e94248940c6dbf6b12c23c/artifacts/ecovadis_model'

In [18]:
# loaded_trainer = mlflow.pyfunc.load_model(model_uri)
loaded_trainer = mlflow.pyfunc.load_model('s3://mlflow/6/761906b692e94248940c6dbf6b12c23c/artifacts/ecovadis_model')

In [19]:
loaded_trainer.unwrap_python_model()

In [21]:
# mlflow.statsmodels.save_model(loaded_trainer, 'data/model')#, serialization_format='pickle')
mlflow.pyfunc.save_model("data/model", python_model=loaded_trainer.unwrap_python_model())

2024/06/18 21:31:34 INFO mlflow.types.utils: Unsupported type hint: <class 'pandas.core.frame.DataFrame'>, skipping schema inference
2024/06/18 21:31:34 INFO mlflow.types.utils: Unsupported type hint: <class 'pandas.core.frame.DataFrame'>, skipping schema inference
2024/06/18 21:31:37 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: data/model, flavor: python_function). Fall back to return ['cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback. 


In [22]:
loaded_trainer.predict(df_pd.iloc[:2].drop(columns=["Exited"]))

,Exited
0,-9.958064
1,-12.560959


### Downloaded/Served Trainer
* i.e. testing [model locally](https://mlflow.org/docs/latest/deployment/deploy-model-to-kubernetes/tutorial.html?highlight=kserve#step-6-testing-model-serving-locally)

In [23]:
model_uri

's3://mlflow/6/761906b692e94248940c6dbf6b12c23c/artifacts/ecovadis_model'

In [24]:
! mlflow models serve -m "s3://mlflow/6/761906b692e94248940c6dbf6b12c23c/artifacts/ecovadis_model" --env-manager local -p 5000

2024/06/18 21:32:14 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2024/06/18 21:32:15 INFO mlflow.pyfunc.backend: === Running command 'exec gunicorn --timeout=60 -b 127.0.0.1:5000 -w 1 ${GUNICORN_CMD_ARGS} -- mlflow.pyfunc.scoring_server.wsgi:app'
[2024-06-18 21:32:15 +0000] [1380] [INFO] Starting gunicorn 22.0.0
[2024-06-18 21:32:15 +0000] [1380] [INFO] Listening at: http://127.0.0.1:5000 (1380)
[2024-06-18 21:32:15 +0000] [1380] [INFO] Using worker: sync
[2024-06-18 21:32:15 +0000] [1381] [INFO] Booting worker with pid: 1381
^C
[2024-06-18 21:36:45 +0000] [1380] [INFO] Handling signal: int
[2024-06-18 21:36:45 +0000] [1381] [INFO] Worker exiting (pid: 1381)


output from terminal:

```
root@jupyter-5uperpalo:~/assignment/ecovadis_assignment# curl -X POST -H "Content-Type:application/json" --data '{"inputs": {"CustomerId": [15787619, 15770309], "CreditScore": [844, 656], "Country": ["France", "France"], "Gender": ["Male", "Male"], "Age": [18, 18], "Tenure": [2, 10], "Balance (EUR)": [160980.03, 151762.74], "NumberOfProducts": [1, 1], "HasCreditCard": ["0", "0"], "IsActiveMember": ["0", "1"], "EstimatedSalary": [145936.28, 127014.32], "CustomerFeedback_sentiment3": ["neutral", "neutral"], "CustomerFeedback_sentiment5": ["4 stars", "1 star"], "Surname_Country": ["Taiwan", "United States"], "Surname_Country_region": ["Asia", "Americas"], "Surname_Country_subregion": ["Eastern Asia", "Northern America"], "Country_region": ["Europe", "Europe"], "Country_subregion": ["Western Europe", "Western Europe"], "is_native": ["0", "0"], "Country_hemisphere": ["northern", "northern"], "Country_gdp_per_capita": [57594.03402, 57594.03402], "Country_IncomeGroup": ["High income", "High income"], "Surname_Country_gdp_per_capita": [32756.0, 76329.58227], "Surname_Country_IncomeGroup": ["None", "High income"], "working_class": ["working_age", "working_age"], "stage_of_life": ["teen", "teen"], "generation": ["gen_z", "gen_z"]}}' http://127.0.0.1:5000/invocations
{"predictions": [{"Exited": -9.958063834023024}, {"Exited": -12.560959268044444}]}
```

# Model deployment using Kserve + Knative + Istio

**reasons:**
* Kserve is less mature than Seldon core but is primarly adopted by Kubeflow and MLflow
* "Serverless installation using Knative" is a primary installation option of Ksserve as *KServe Serverless installation enables autoscaling based on request volume and supports scale down to and from zero. It also supports revision management and canary rollout based on revisions.*
* Istio is a primary choice of service routing for Knative
* Kserve uses MLserver from Seldoncore but according to "quick internet search" it is adopted more by community than Seldon-core as Seldon-core focuses more on enterprise version of its software 

**issues:**
* Kserve uses MLserver which ignores `requirements.txt` and needs prepackaged conda environment in tar.gz file
* Kserve uses old version of MLserver(1.3.2) resulting in incorrect usage of `conda-unpack`
* related code has to be included with model and conda environment  
* mlserver with correct version has to be defined in the conda env:
```
  - mlserver==1.3.5
  - mlserver-mlflow==1.3.5
```

**related github issue discussions/troubleshooting:**
* https://github.com/kserve/kserve/issues/3733#issuecomment-2168261646
* https://github.com/SeldonIO/MLServer/issues/1801

## Create conda environment, tar.gz it and upload it to Minio

In [7]:
from minio import Minio

# to download conda.yaml
client = Minio("10.152.183.156:9000", "minioadmin", "minioadmin", secure=False)
bucket = "mlflow"
object_name = "6/761906b692e94248940c6dbf6b12c23c/artifacts/ecovadis_model/conda.yaml"
file_path = "notebooks/conda.yaml"
client.fget_object(bucket_name=bucket, object_name=object_name, file_path=file_path)

following commands were run in terminal to create environment:
```
pip install conda_pack
conda env create --force -f conda.yaml
conda-pack --name mlflow-env --output environment.tar.gz --force --ignore-editable-packages --ignore-missing-files
```

In [28]:
# upload environmnet.tar.gz
object_name = "6/761906b692e94248940c6dbf6b12c23c/artifacts/ecovadis_model/environment.tar.gz"
file_path = "notebooks/environment.tar.gz"
client.fput_object(bucket_name=bucket, object_name=object_name, file_path=file_path)

## Create Kserver Inference Service in Kubernetes
```
apiVersion: "serving.kserve.io/v1beta1"
kind: "InferenceService"
metadata:
  name: "custom-ecovadis-model"
  namespace: "mlflow-kserve-success6g"
spec:
  predictor:
    serviceAccountName: success6g
    model:
      modelFormat:
        name: mlflow
      protocolVersion: v2
      storageUri: "s3://mlflow/6/761906b692e94248940c6dbf6b12c23c/artifacts/ecovadis_model"
```

## Test the Inference service

```
export INGRESS_NAME=istio-ingressgateway
export INGRESS_NS=istio-system
export INGRESS_PORT=$(kubectl -n "${INGRESS_NS}" get service "${INGRESS_NAME}" -o jsonpath='{.spec.ports[?(@.name=="http2")].nodePort}')
export INGRESS_HOST=$(kubectl get po -l istio=ingressgateway -n "${INGRESS_NS}" -o jsonpath='{.items[0].status.hostIP}')
export SERVICE_HOSTNAME=$(kubectl get inferenceservice custom-ecovadis-model -n mlflow-kserve-success6g -o jsonpath='{.status.url}' | cut -d "/" -f 3)

curl -v \
  -H "Host: ${SERVICE_HOSTNAME}" \
  -H "Content-Type: application/json" \
  -d @./test.json \
  http://${INGRESS_HOST}:${INGRESS_PORT}/v2/models/custom-ecovadis-model/infer
```

## SUMMARY 

This unfortunately crashes due to the size of conda environment that is being downloaded from minio database by Kserve. COnda env has 3.36GB due to language models (libraries). 

**Note:** this approach was succesfully tested on a custom model that used small conda env ~360MB

## WORKAROUND

1. save/serve(deploy) only the LightGBM model 
2. preprocess the data locally; this could be done in production using (i) [Kserve InferenceGpraph](https://kserve.github.io/website/master/modelserving/inference_graph/#concepts) or (ii) [pipelines in Kubeflow](https://www.kubeflow.org/docs/components/pipelines/v1/sdk/build-pipeline/)
3. send preprocessed data to API

In [15]:
run_name = "init_run"
with mlflow.start_run(experiment_id=experiment_id, run_name=run_name, nested=True) as run:
    mlflow.log_params(trainer.optimizer.best)
    mlflow.log_metrics(metrics_dict_flattened)

    # Log tags
    mlflow.set_tags(
        tags={
            "project": "ecovadis",
            "optimizer_engine": "optuna",
            "model_family": "ligtgbm",
            "feature_set_version": 1,
        }
    )
    # Log figure - for future fun
    # mlflow.log_figure(figure=correlation_plot, artifact_file="correlation_plot.png")

    artifact_path = "ecovadis_raw_model"
    registered_model_name = "ecovadis_raw_lgbm_model"
    mlflow.lightgbm.log_model(
            lgb_model=trainer.model,
            artifact_path=artifact_path, 
            registered_model_name=registered_model_name)
    # mlflow.pyfunc.log_model(python_model=trainer, 
    #                         artifact_path=artifact_path, 
    #                         registered_model_name=registered_model_name,
    #                         code_paths=["churn_pred"])  
    model_uri = mlflow.get_artifact_uri(artifact_path)
    print(f"Run ID:\n{run.info.run_id}\nModel uri:\n{model_uri}")

Successfully registered model 'ecovadis_raw_lgbm_model'.
2024/06/19 05:43:13 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ecovadis_raw_lgbm_model, version 1


Run ID:
9a74f8e9087044b9b9d4935a268c26d0
Model uri:
s3://mlflow/6/9a74f8e9087044b9b9d4935a268c26d0/artifacts/ecovadis_raw_model


Created version '1' of model 'ecovadis_raw_lgbm_model'.


In [39]:
df_infser_test = df_pd.iloc[:2]
for prep in trainer.preprocessors:
    df_infser_test = prep.transform(df_infser_test)
    if hasattr(trainer, "optimizer"):
        if hasattr(trainer.optimizer, "best_to_drop"):  # type: ignore
            df_infser_test.drop(
                columns=trainer.optimizer.best_to_drop,  # type: ignore
                inplace=True,
            )
df_infser_test = df_infser_test.drop(columns=trainer.id_cols + [trainer.target_col])

In [40]:
df_infser_test.transpose().to_dict(orient='index')

{'is_native': {0: 1.0, 1: 1.0},
 'generation': {0: 1.0, 1: 1.0},
 'Surname_Country': {0: 1.0, 1: 2.0},
 'Country_subregion': {0: 1.0, 1: 1.0},
 'Surname_Country_subregion': {0: 1.0, 1: 2.0},
 'Surname_Country_IncomeGroup': {0: 1.0, 1: 2.0},
 'stage_of_life': {0: 1.0, 1: 1.0},
 'IsActiveMember': {0: 1.0, 1: 2.0},
 'Country': {0: 1.0, 1: 1.0},
 'CustomerFeedback_sentiment5': {0: 1.0, 1: 2.0},
 'CustomerFeedback_sentiment3': {0: 1.0, 1: 1.0},
 'Gender': {0: 1.0, 1: 1.0},
 'Surname_Country_region': {0: 1.0, 1: 2.0},
 'working_class': {0: 1.0, 1: 1.0},
 'HasCreditCard': {0: 1.0, 1: 1.0},
 'EstimatedSalary': {0: 145936.28125, 1: 127014.3203125},
 'Age': {0: 18.0, 1: 18.0},
 'Tenure': {0: 2.0, 1: 10.0},
 'NumberOfProducts': {0: 1.0, 1: 1.0},
 'Country_gdp_per_capita': {0: 57594.03515625, 1: 57594.03515625},
 'Surname_Country_gdp_per_capita': {0: 32756.0, 1: 76329.5859375},
 'CreditScore': {0: 844.0, 1: 656.0},
 'Balance (EUR)': {0: 160980.03125, 1: 151762.734375}}

create test.json:
```
cat test.json 
{
  "parameters": {
    "content_type": "pd"
  },
  "inputs": [    
      {
        "name": "is_native",
        "shape": [1],
        "datatype": "FP64",
        "data": [1.0]
      },
      {
        "name": "generation",
        "shape": [1],
        "datatype": "FP64",
        "data": [1.0]
      },
      {
        "name": "Surname_Country",
        "shape": [1],
        "datatype": "FP64",
        "data": [1.0]
      },
      {
        "name": "Country_subregion",
        "shape": [1],
        "datatype": "FP64",
        "data": [1.0]
      },
      {
        "name": "Surname_Country_subregion",
        "shape": [1],
        "datatype": "FP64",
        "data": [1.0]
      },
      {
        "name": "Surname_Country_IncomeGroup",
        "shape": [1],
        "datatype": "FP64",
        "data": [1.0]
      },
      {
        "name": "stage_of_life",
        "shape": [1],
        "datatype": "FP64",
        "data": [1.0]
      },
      {
        "name": "IsActiveMember",
        "shape": [1],
        "datatype": "FP64",
        "data": [1.0]
      },
      {
        "name": "Country",
        "shape": [1],
        "datatype": "BYTES",
        "data": [1.0]
      },
      {
        "name": "CustomerFeedback_sentiment5",
        "shape": [1],
        "datatype": "FP64",
        "data": [1.0]
      },
      {
        "name": "CustomerFeedback_sentiment3",
        "shape": [1],
        "datatype": "FP64",
        "data": [1.0]
      },
      {
        "name": "Gender",
        "shape": [1],
        "datatype": "FP64",
        "data": [1.0]
      },
      {
        "name": "Surname_Country_region",
        "shape": [1],
        "datatype": "FP64",
        "data": [1.0]
      },
      {
        "name": "working_class",
        "shape": [1],
        "datatype": "FP64",
        "data": [1.0]
      },
      {
        "name": "HasCreditCard",
        "shape": [1],
        "datatype": "FP64",
        "data": [1.0]
      },
      {
        "name": "EstimatedSalary",
        "shape": [1],
        "datatype": "FP64",
        "data": [145936.28]
      },
      {
        "name": "Age",
        "shape": [1],
        "datatype": "FP64",
        "data": [18]
      },
      {
        "name": "Tenure",
        "shape": [1],
        "datatype": "FP64",
        "data": [2]
      },
      {
        "name": "NumberOfProducts",
        "shape": [1],
        "datatype": "FP64",
        "data": [1.0]
      },
      {
        "name": "Country_gdp_per_capita",
        "shape": [1],
        "datatype": "FP64",
        "data": [57594.03402]
      },
      {
        "name": "Surname_Country_gdp_per_capita",
        "shape": [1],
        "datatype": "FP64",
        "data": [32756.0]
      },
      {
        "name": "CreditScore",
        "shape": [1],
        "datatype": "FP64",
        "data": [844.0]
      },
      {
        "name": "Balance (EUR)",
        "shape": [1],
        "datatype": "FP64",
        "data": [160980.03125]
      }
  ]

```

test service:
```
export INGRESS_NAME=istio-ingressgateway
export INGRESS_NS=istio-system
export SERVICE_HOSTNAME=custom-ecovadis-raw-model.mlflow-kserve-success6g.example.com
export INGRESS_PORT=$(kubectl -n "${INGRESS_NS}" get service "${INGRESS_NAME}" -o jsonpath='{.spec.ports[?(@.name=="http2")].nodePort}')
export SERVICE_HOSTNAME=$(kubectl get inferenceservice custom-ecovadis-raw-model -n mlflow-kserve-success6g -o jsonpath='{.status.url}' | cut -d "/" -f 3)

curl -v \
  -H "Host: ${SERVICE_HOSTNAME}" \
  -H "Content-Type: application/json" \
  -d @./test.json \
  http://${INGRESS_HOST}:${INGRESS_PORT}/v2/models/custom-ecovadis-raw-model/infer
```
output:
```
{INGRESS_HOST}:${INGRESS_PORT}/v2/models/custom-ecovadis-raw-model/infer
*   Trying 10.1.24.50:32702...
* Connected to 10.1.24.50 (10.1.24.50) port 32702 (#0)
> POST /v2/models/custom-ecovadis-raw-model/infer HTTP/1.1
> Host: custom-ecovadis-raw-model.mlflow-kserve-success6g.example.com
> User-Agent: curl/8.1.1
> Accept: */*
> Content-Type: application/json
> Content-Length: 2823
> 
< HTTP/1.1 200 OK
< ce-endpoint: custom-ecovadis-raw-model
< ce-id: 0cd6e630-3a2b-44de-8284-299a10cf5283
< ce-inferenceservicename: mlserver
< ce-modelid: custom-ecovadis-raw-model
< ce-namespace: mlflow-kserve-success6g
< ce-requestid: 0cd6e630-3a2b-44de-8284-299a10cf5283
< ce-source: io.seldon.serving.deployment.mlserver.mlflow-kserve-success6g
< ce-specversion: 0.3
< ce-type: io.seldon.serving.inference.response
< content-length: 251
< content-type: application/json
< date: Wed, 19 Jun 2024 06:49:46 GMT
< server: istio-envoy
< x-envoy-upstream-service-time: 712
< 
* Connection #0 to host 10.1.24.50 left intact
{"model_name":"custom-ecovadis-raw-model","id":"0cd6e630-3a2b-44de-8284-299a10cf5283","parameters":{"content_type":"np"},"outputs":[{"name":"output-1","shape":[1,1],"datatype":"FP64","parameters":{"content_type":"np"},"data":[-9.958064]}]}
```